In [2]:
import cirq
import cirq_google

In [4]:
import numpy as np

def main():
    # Create circuit.
    circuit = make_bell_test_circuit()
    print('Circuit:')
    print(circuit)

    # Run simulations.
    print()
    repetitions = 75
    print(f'Simulating {repetitions} repetitions...')
    result = cirq.Simulator().run(program=circuit, repetitions=repetitions)

    # Collect results.
    a = np.array(result.measurements['a'][:, 0])
    b = np.array(result.measurements['b'][:, 0])
    x = np.array(result.measurements['x'][:, 0])
    y = np.array(result.measurements['y'][:, 0])
    outcomes = a ^ b == x & y
    win_percent = len([e for e in outcomes if e]) * 100 / repetitions

    # Print data.
    print()
    print('Results')
    print('a:', bitstring(a))
    print('b:', bitstring(b))
    print('x:', bitstring(x))
    print('y:', bitstring(y))
    print('(a XOR b) == (x AND y):\n  ', bitstring(outcomes))
    print(f'Win rate: {win_percent}%')


def make_bell_test_circuit():
    alice = cirq.GridQubit(0, 0)
    bob = cirq.GridQubit(1, 0)
    alice_referee = cirq.GridQubit(0, 1)
    bob_referee = cirq.GridQubit(1, 1)

    circuit = cirq.Circuit()

    # Prepare shared entangled state.
    circuit.append(
        [
            cirq.H(alice),
            cirq.CNOT(alice, bob),
        ]
    )

    # Referees flip coins.
    circuit.append(
        [
            cirq.H(alice_referee),
            cirq.H(bob_referee),
        ]
    )

    # Players do a sqrt(X) based on their referee's coin.
    circuit.append(
        [
            cirq.X(alice) ** -0.25,
            cirq.CNOT(alice_referee, alice) ** 0.5,
            cirq.CNOT(bob_referee, bob) ** 0.5,
        ]
    )

    # Then results are recorded.
    circuit.append(
        [
            cirq.measure(alice, key='a'),
            cirq.measure(bob, key='b'),
            cirq.measure(alice_referee, key='x'),
            cirq.measure(bob_referee, key='y'),
        ]
    )

    return circuit


def bitstring(bits):
    return ''.join('1' if e else '_' for e in bits)


if __name__ == '__main__':
    main()

Circuit:
(0, 0): ───H───@───X^-0.25───X────────M('a')───
               │             │
(0, 1): ───H───┼─────────────@^0.5────M('x')───
               │
(1, 0): ───────X───X─────────M('b')────────────
                   │
(1, 1): ───H───────@^0.5─────M('y')────────────

Simulating 75 repetitions...

Results
a: 1_11___111111___1_111___11_1__1_1___111_1__1111_11_____1111111__1__1__1___1
b: 1__1__1_1_1111_1__1_1_1_11_11_11______1_11___11111111__1_11111_11__1_11_1__
x: 1__1__1_11____111_11________11_11__111_____11__1_1111__1___1__________1___1
y: ____1_11_111___11__1111_11111_1_1_1_11___11111_11__1_____11_111_111_1_1_1_1
(a XOR b) == (x AND y):
   11_1111_11111_11111111_11111111_111111111_11111111_1_111_111111_11111__1_11
Win rate: 82.66666666666667%
